## Bayes' Theorem Calculator

Whenever you are using a test to determine if something is happening or not, it is time to use Bayes' theorem to avoid comitting the base rate fallacy. However, the mathematical notation and formula may be intimidating. You can use the following calculator to learn how Bayes' theorem works, and to answer practical questions about probability.

Run the following box by clicking on it and pressing `Shift+Return`.

In [ ]:
from base_rate import *
%matplotlib widget

Run the following box.

In [ ]:
BayesForm();

In [13]:
from IPython.display import HTML, Javascript

In [206]:
display(HTML('''<style>
#sliders {
    display: flex;
    flex-direction: row;
    flex-wrap: wrap;
    width: 100%;
}

#slider1 {
    flex: 33.33%;
}

#slider2 {
    flex: 33.33%;
}

#slider3 {
    flex: 33.33%;
}

.slider text {
    font-size: 14pt;
}

.slider_label {
    font-size: 14pt;
    text-align: center;
}

#graphic {
    width: 100%;
}
</style>'''))
Javascript(filename='base_rate.js')

<IPython.core.display.Javascript object>

In [207]:
Javascript("""
 (function(element){
     require(['viz'], function(viz) {
          viz(element.get(0))
     });
 })(element);
 """)

<IPython.core.display.Javascript object>